# Cube dual volume computation

In this attempt we only use duals of the $d-1$-simplices for computing volumes as opposed to the duals of the $d$-simplices

We benchmark different methods for compute dual volumes on a simple triagulated cube (found using hyperct

In [ ]:



# ddg imports
import os, sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from hyperct import *
from ddgclib._plotting import *
from ddgclib._sphere import *


def polytope_volume(cutting_planes):
    """
    Computes the volume of a convex polytope defined by its linear cutting planes.
    
    Parameters:
    cutting_planes (ndarray): An array of shape (m, d + 1), where m is the number of cutting planes and d is the dimension of the polytope. Each row of the array corresponds to a cutting plane in the form [a_1, a_2, ..., a_d, b], where a_i are the coefficients of the hyperplane equation and b is the constant term.
    
    Returns:
    float: The volume of the polytope.
    """
    # Create an instance of HalfspaceIntersection with the cutting planes
    hs = HalfspaceIntersection(cutting_planes[:,:-1], -cutting_planes[:,-1])
    
    # Compute the volume of the polytope using the HalfspaceIntersection object
    volume = hs.volume
    
    return volume


def polytope_volume_gram(vertices):
    """
    Computes the volume of a convex polytope defined by its vertices using Gram's relation.
    
    Parameters:
    vertices (ndarray): An array of shape (n, d), where n is the number of vertices and d is the dimension of the polytope.
    
    Returns:
    float: The volume of the polytope.
    """
    # Calculate the Gram matrix
    gram_matrix = np.dot(vertices.T, vertices)
    
    # Compute the determinant of the Gram matrix
    det_gram_matrix = np.linalg.det(gram_matrix)
    
    # Compute the volume of the polytope using Gram's relation
    volume = np.sqrt(np.abs(det_gram_matrix)) / np.math.factorial(vertices.shape[0])
    
    return volume


In [31]:


def circumcenter_old(triangle):
    """
    Compute the circumcenter of a triangle in 3D space given the vertex positions.
    
    Arguments:
    triangle -- a numpy array of shape (3,3) representing the vertex positions of the triangle
    
    Returns:
    circumcenter -- a numpy array of shape (3,) representing the circumcenter of the triangle
    """
    
    # Calculate the edge vectors of the triangle
    edge_ab = triangle[1] - triangle[0]
    edge_ac = triangle[2] - triangle[0]
    
    # Calculate the normal vector of the triangle
    normal = np.cross(edge_ab, edge_ac)
    
    # Calculate the midpoints of the edges
    midpoint_ab = (triangle[0] + triangle[1]) / 2
    midpoint_ac = (triangle[0] + triangle[2]) / 2
    
    # Calculate the projection of the midpoints onto the normal vector
    projection_ab = np.dot(midpoint_ab - triangle[0], normal) / np.dot(normal, normal) * normal
    projection_ac = np.dot(midpoint_ac - triangle[0], normal) / np.dot(normal, normal) * normal
    
    # Calculate the intersection of the two projections
    circumcenter = np.linalg.solve(np.array([[-edge_ab[0], edge_ac[0]], 
                                             [-edge_ab[1], edge_ac[1]]]), 
                                    np.array([projection_ab[0] - projection_ac[0], 
                                              projection_ab[1] - projection_ac[1]]))
    #circumcenter += triangle[0]
    
    return circumcenter

def circumcenter2_old(triangle):
    """
    Compute the circumcenter of a triangle in 3D space given the vertex positions.
    
    Arguments:
    triangle -- a numpy array of shape (3,3) representing the vertex positions of the triangle
    
    Returns:
    circumcenter -- a numpy array of shape (3,) representing the circumcenter of the triangle
    """
    
    # Calculate the edge vectors of the triangle
    edge_ab = triangle[1] - triangle[0]
    edge_ac = triangle[2] - triangle[0]
    
    # Calculate the normal vector of the triangle
    normal = np.cross(edge_ab, edge_ac)
    
    # Pre-compute some values for efficiency
    normal_dot = np.dot(normal, normal)
    midpoint_diff = (triangle[0] + triangle[2]) / 2 - (triangle[0] + triangle[1]) / 2
    
    # Calculate the projection of the midpoints onto the normal vector
    projection_ab = np.dot(midpoint_diff + edge_ab / 2, normal) / normal_dot * normal
    projection_ac = np.dot(midpoint_diff - edge_ab / 2, normal) / normal_dot * normal
    
    # Calculate the intersection of the two projections
    circumcenter = np.linalg.solve(np.array([[-edge_ab[0], edge_ac[0]], 
                                             [-edge_ab[1], edge_ac[1]]]), 
                                    np.array([projection_ab[0] - projection_ac[0], 
                                              projection_ab[1] - projection_ac[1]]))
    #circumcenter += triangle[0]
    
    return circumcenter


In [32]:
import numpy as np

def distance(a, b):
    return np.linalg.norm(a - b)

def circumcenter(vertices):
    A, B, C = vertices
    a = distance(B, C)
    b = distance(C, A)
    c = distance(A, B)

    if a == 0 or b == 0 or c == 0:
        raise ValueError("Invalid triangle: degenerate or duplicate vertices")

    # Calculate the circumradius
    s = (a + b + c) / 2
    area = np.sqrt(s * (s - a) * (s - b) * (s - c))
    circumradius = (a * b * c) / (4 * area)

    # Calculate the normalized barycentric coordinates
    alpha = (a * a * (b * b + c * c - a * a)) / (a * a + b * b + c * c)
    beta = (b * b * (c * c + a * a - b * b)) / (a * a + b * b + c * c)
    gamma = (c * c * (a * a + b * b - c * c)) / (a * a + b * b + c * c)

    circumcenter = (alpha * A + beta * B + gamma * C) / (alpha + beta + gamma)

    return circumcenter

# Example usage
triangle_vertices = np.array([[1, 2, 0], [4, 0, 1], [0, 0, 5]], dtype=np.float64)
print("Circumcenter:", circumcenter(triangle_vertices))

Circumcenter: [1.5 0.5 2.5]


This program defines a function circumcenter that takes an array of the triangle vertices' positions and returns the circumcenter's coordinates. The function calculates the distances between each pair of vertices and uses them to compute the normalized barycentric coordinates, which are then used to find the circumcenter's coordinates.

In [33]:
points = np.random.rand(3, 3)
points = np.array([[1, 0, 0],
                   [0, 1, 0],
                   [0, 0, 0]])
triangle = points
points

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 0]])

In [34]:
cd =  circumcenter(points)
cd

array([0.5, 0.5, 0. ])

In [35]:
circumcenter

<function __main__.circumcenter(vertices)>

In [36]:
cd

array([0.5, 0.5, 0. ])

In [37]:
HC.Vd = VertexCacheField()  # TODO: Add arguments

In [38]:
HC.Vd, HC.V

(<ddgclib._vertex.VertexCacheField at 0x7f8cfa8b43a0>,
 <ddgclib._vertex.VertexCacheIndex at 0x7f8cfa8afe50>)

In [39]:
VertexCacheField

ddgclib._vertex.VertexCacheField

In [40]:
for v in HC.V:
        v.vd = set()
            
            
for v1 in HC.V:
    for v2 in v1.nn:
        v1nn_u_v2nn = None

In [41]:
v1.nn.intersection(v2.nn),v2.nn

({<ddgclib._vertex.VertexCube at 0x7f8cfa4eb400>,
  <ddgclib._vertex.VertexCube at 0x7f8cfa8aee30>},
 {<ddgclib._vertex.VertexCube at 0x7f8cfa4eb400>,
  <ddgclib._vertex.VertexCube at 0x7f8cfa8affa0>})

In [42]:
np.zeros([3,3])

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [43]:
verts = np.zeros([3, 3])
verts[0] = v.x_a
verts

array([[0.125, 0.5  , 0.875],
       [0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   ]])

In [44]:
# Write a function that builds a dual cache called HC.Vd,
# each member of the `Vd` set should be connect only to primal
# vertices in HC.V
def compute_vd(HC):
    """
    Computes the dual vertices of a primal vertex cache HC.V on
    each dim - 1 simplex.
    
    Currently only dim = 3 is supported
    """
    # Construct dual cache
    HC.Vd = VertexCacheField() 
    
    # Construct dual neighbour sets
    for v in HC.V:
        v.vd = set()
            
    #hcv = copy.copy(HC.V)        
    for v1 in HC.V:
        for v2 in v1.nn:
            # Find all v2.nn also connected to v1:
            v1nn_u_v2nn = v1.nn.intersection(v2.nn)
            for v3 in v1nn_u_v2nn:
                # TODO: Re-implement cache:
                verts = np.zeros([3, 3])
                verts[0] = v1.x_a
                verts[1] = v2.x_a
                verts[2] = v3.x_a
                # Compute the circumcentre:
                cd = circumcenter(verts)
                vd = HC.Vd[tuple(cd)]
                # Connect to all primal vertices
                for v in [v1, v2, v3]:
                    v.vd.add(vd)
                    vd.nn.add(v)
                    
    return HC  # self

# Complex 
bounds = [(0, 1),] * 3  # 4**2  = 16 m3
HC = Complex(3, domain=bounds)
HC.triangulate()
if 0:  # randomize
    hcv = copy.copy(HC.V)
    for v in HC.V:
        xnew = tuple(v.x_a + 0.2*(np.random.rand(3)-0.5))
        HC.V.move(v, xnew)
##
HC = compute_vd(HC)


In [45]:
for vd in HC.Vd:
    #print(f'vd.x = {vd.x}')
    vdp = HC.V[vd.x]
    for vp in vd.nn:
        #print(vp)
        vd.connect(vp)
        vdp.connect(vp)

In [ ]:
plot_polyscope(HC)
ps.show()

In [80]:
rr = 0.1*(np.random.rand(3)-0.5)
rr

array([ 0.02546683, -0.04825737,  0.04204589])

In [83]:
#v.move()
xnew = tuple(v.x_a + 0.1*(np.random.rand(3)-0.5))
HC.V.move(v, xnew)

In [85]:
bounds = [(0, 1),] * 3  # 4**2  = 16 m3
HC = Complex(3, domain=bounds)
HC.triangulate()

hcv = copy.copy(HC.V)
for v in HC.V:
    xnew = tuple(v.x_a + 0.2*(np.random.rand(3)-0.5))
    HC.V.move(v, xnew)

plot_polyscope(HC)
ps.show()

In [22]:
bounds = [(-1, 3),] * 3  # 4**2  = 16 m3
HC = Complex(3, domain=bounds)
for p in points:
    HC.V[tuple(p)]
    
for v1 in HC.V:
    for v2 in HC.V:
        v1.connect(v2)
        
cd = circumcenter(triangle)
vc = HC.V[tuple(cd)]
for v1 in HC.V:
    vc.connect(v1)

In [9]:
HC.plot_complex()
plt.show()

/home/stefan_endres/projects/ddg/ddgclib/ddgclib/_complex.py:2832: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig_complex.show()


<Figure size 640x480 with 0 Axes>

In [10]:
plot_polyscope(HC)
ps.show()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 530.41.03


In [ ]:
from hyperct import *

In [25]:
bounds = [(-1, 3),] * 3  # 4**2  = 16 m3
HC = Complex(3, domain=bounds)

In [26]:
HC.triangulate()
#HC.refine_all()
#HC.refine_all()



In [27]:
plot_polyscope(HC)
ps.show()

In [52]:
HC.V.cache

Va = []
for v in HC.V:
    Va.append(v.x)
    
Va = np.array(X ,dtype='float64')
Va

array([[-1. , -1. , -1. ],
       [ 3. ,  3. ,  3. ],
       [ 3. , -1. , -1. ],
       [-1. ,  3. , -1. ],
       [ 3. ,  3. , -1. ],
       [-1. , -1. ,  3. ],
       [ 3. , -1. ,  3. ],
       [-1. ,  3. ,  3. ],
       [ 1. ,  1. ,  1. ],
       [ 1. , -1. , -1. ],
       [ 1. ,  1. , -1. ],
       [-1. ,  1. , -1. ],
       [ 3. ,  1. , -1. ],
       [ 1. ,  3. , -1. ],
       [ 1. ,  1. ,  3. ],
       [-1. , -1. ,  1. ],
       [ 3. , -1. ,  1. ],
       [-1. ,  3. ,  1. ],
       [ 3. ,  3. ,  1. ],
       [ 1. , -1. ,  1. ],
       [-1. ,  1. ,  1. ],
       [ 3. ,  1. ,  1. ],
       [ 1. ,  3. ,  1. ],
       [ 1. , -1. ,  3. ],
       [-1. ,  1. ,  3. ],
       [ 3. ,  1. ,  3. ],
       [ 1. ,  3. ,  3. ],
       [ 2. ,  0. ,  0. ],
       [ 2. ,  0. ,  2. ],
       [ 0. ,  2. ,  0. ],
       [ 0. ,  0. ,  0. ],
       [ 2. ,  2. ,  2. ],
       [ 0. ,  2. ,  2. ],
       [ 0. ,  0. ,  2. ],
       [ 2. ,  2. ,  0. ],
       [ 2. , -1. , -1. ],
       [ 2. ,  0. , -1. ],
 

In [57]:

X

array([[ 0. , -1. , -1. ],
       [ 0. , -1. ,  1. ],
       [ 0. ,  1. ,  0. ],
       [ 0.5, -0.5,  0.5],
       [ 0.5, -0.5, -0.5],
       [ 0. ,  0. , -1. ],
       [ 0. ,  0. ,  1. ],
       [ 1. ,  0. ,  1. ],
       [ 1. ,  1. ,  0. ],
       [ 1. ,  0. , -1. ],
       [ 0. , -1. ,  0. ],
       [-1. ,  1. ,  0. ],
       [ 1. , -1. ,  0. ],
       [ 0.5,  0.5, -0.5],
       [ 0.5,  0.5,  0.5],
       [-0.5, -0.5, -0.5],
       [-0.5, -0.5,  0.5],
       [-1. ,  0. ,  1. ],
       [-1. ,  0. , -1. ],
       [ 1. ,  0. ,  0. ],
       [-1. , -1. ,  0. ],
       [ 0. ,  1. , -1. ],
       [ 0. ,  1. ,  1. ],
       [-1. ,  0. ,  0. ],
       [-0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5]], dtype=float128)

In [66]:
# Construct cutting_planes

# half spaces are Stacked Inequalities of the form Ax + b <= 0 in format [A; b]
# A = np.hstack((halfspaces[:, :-1], norm_vector))
#b = - halfspaces[:, -1:]


# Contants of hyperplanes are half the norm of the vector:
i = 30
HC.V[tuple(Va[i])]
vi = HC.V[tuple(Va[i])]
Xi = vi.x_a
X = []
for v in vi.nn:
    X.append(v.x)
    
X = np.array(X)

A = X - Xi
A_norm = normalized(A)
b = np.sum(0.5 * A * A_norm, axis=1)  # For some reason no minus factor like formula
A = A_norm

# NOTE: Remove zero vectors
A = A[mask]
b = b[mask]

In [67]:
cutting_planes = np.column_stack((A, -b))
cutting_planes

array([[ 0.        , -0.70710678, -0.70710678, -0.70710678],
       [ 0.        , -0.70710678,  0.70710678, -0.70710678],
       [ 0.        ,  1.        ,  0.        , -0.5       ],
       [ 0.57735027, -0.57735027,  0.57735027, -0.4330127 ],
       [ 0.57735027, -0.57735027, -0.57735027, -0.4330127 ],
       [ 0.        ,  0.        , -1.        , -0.5       ],
       [ 0.        ,  0.        ,  1.        , -0.5       ],
       [ 0.70710678,  0.        ,  0.70710678, -0.70710678],
       [ 0.70710678,  0.70710678,  0.        , -0.70710678],
       [ 0.70710678,  0.        , -0.70710678, -0.70710678],
       [ 0.        , -1.        ,  0.        , -0.5       ],
       [-0.70710678,  0.70710678,  0.        , -0.70710678],
       [ 0.70710678, -0.70710678,  0.        , -0.70710678],
       [ 0.57735027,  0.57735027, -0.57735027, -0.4330127 ],
       [ 0.57735027,  0.57735027,  0.57735027, -0.4330127 ],
       [-0.57735027, -0.57735027, -0.57735027, -0.4330127 ],
       [-0.57735027, -0.

In [68]:
from scipy.spatial import HalfspaceIntersection

#Xi  # Current interior point
halfspaces = cutting_planes
hs = HalfspaceIntersection(halfspaces, Xi)
hs.dual_volume

28.444444444444436

In [ ]:
# # Compute the volume of the polytope using the HalfspaceIntersection object
 #   volume = hs.volume

In [44]:
X[i] 

array([0., 0., 0.])